In [1]:
import os
from json import load
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

import chainer
import chainer.links as L
import chainer.functions as F
from chainer import cuda
from chainer import optimizers
from chainer import iterators
from chainer import training
from chainer.training import extensions
from chainer import datasets
from chainer.datasets import TransformDataset
from chainer.datasets import LabeledImageDataset
from functools import partial
from chainercv import transforms

In [2]:
train = LabeledImageDataset('/home/ubuntu/intern-winter-2019/dataset/simple_crop/train_data/train_label.csv',
                            '/home/ubuntu/intern-winter-2019/dataset/simple_crop/train_data/images/',dtype=np.float32)
test = LabeledImageDataset('/home/ubuntu/intern-winter-2019/dataset/simple_crop/test_data/test_label.csv',
                            '/home/ubuntu/intern-winter-2019/dataset/simple_crop/test_data/images/',dtype=np.float32)

In [3]:
def transform(data, train='False'):
    img, label = data
    img = img.copy()

    # Color augmentation
    if train:
        img = transforms.pca_lighting(img, 76.5)

    # Random flip & crop
    if train:
        img = transforms.random_flip(img, x_random=True)
        img = transforms.random_expand(img, max_ratio=1.5)
        img = transforms.random_crop(img, (200, 200))
    img=np.array(img, dtype=np.float32)
    img=L.model.vision.resnet.prepare(img)

    return img, label

In [4]:
train_dataset = TransformDataset(train, partial(transform, train=True))
test_dataset = TransformDataset(test, partial(transform, train=False))

In [ ]:
class Model(chainer.Chain):
    def __init__(self, dim=2):
        super(Model, self).__init__()
        with self.init_scope():
            self.res50=L.ResNet50Layers(None)
            self.fc1=L.Linear(None, 2)
            self.fc=L.Linear(None, dim)
            
    def __call__(self, x, train=False):
        with chainer.using_config('train', train):
            h=self.res50(x, layers=['fc6'])['fc6']
            h=self.fc1(h)
            h=F.relu(h)
            h=F.normalize(h)*50
            y=self.fc(h)
        return y
    
class SoftMaxEntoropyLoss(chainer.Chain):
    def __init__(self, model):
        super(SoftMaxEntoropyLoss, self).__init__()
        with self.init_scope():
            self.model = model

    def __call__(self, x, t):
        y = self.model(x)
        loss = F.softmax_cross_entropy(y, t)
        accuracy = F.accuracy(y, t)
        summary = F.classification_summary(y, t, beta = 1.0)
        precision = summary[0]
        recall = summary[1]
        f_value = summary[2]
        precision_dict=dict(('precision_%d' % i, val) for i, val in enumerate(summary[0]))
        recall_dict=dict(('recall_%d' % i, val) for i, val in enumerate(summary[1]))
        f_value_dict=dict(('f_value_%d' % i, val) for i, val in enumerate(summary[2]))
        main_dict={'loss':loss, 'accuracy':accuracy}
        main_dict.update(precision_dict)
        main_dict.update(recall_dict)
        main_dict.update(f_value_dict)
        chainer.report(main_dict, self)
        return loss

In [6]:
# ResNet50の初期化 modelの作成
res50 = L.ResNet50Layers(None)

In [7]:
model=Model()
loss=SoftMaxEntoropyLoss(model)

In [12]:
n_epoch = 15
batchsize = 32
out_dir = './result/'
report_interval = (10, 'iteration')

train_iter = iterators.MultithreadIterator\
(train_dataset, batchsize, repeat=True, shuffle=True)
test_iter = iterators.MultithreadIterator\
(test_dataset, batchsize, repeat=False, shuffle=False)

In [13]:
#fine tuningなのでMomentumSDG
optimizer = chainer.optimizers.MomentumSGD(lr=1e-6)
optimizer.setup(loss)

In [14]:
#device=0でGPU, device=-1でCPU
updater = training.StandardUpdater(train_iter, optimizer, device=0)

In [15]:
trainer = training.Trainer(updater, (n_epoch, 'epoch'), out=out_dir)

trainer.extend(extensions.LogReport(trigger=report_interval))
trainer.extend(extensions.PrintReport(['epoch', 
                                       'iteration', 
                                       'main/loss',
                                       'main/accuracy', 
                                       'main/precision_0', 
                                       'main/precision_1',
                                       'main/recall_0',
                                       'main/recall_1']),
               trigger=report_interval, file_name='resnet50_log')
trainer.extend(extensions.PlotReport(y_keys='main/loss', trigger=report_interval, file_name='resnet50_loss.png'))
trainer.extend(extensions.PlotReport(y_keys='main/accuracy', trigger=report_interval, file_name='resnet50_accuracy.png'))

In [ ]:
trainer.run()

epoch       iteration   main/loss   main/accuracy  main/precision_0  main/precision_1  main/recall_0  main/recall_1
0           10          0.376642    0.8625         0.125952          0.960769          nan            0.891105       
0           20          0.377323    0.84375        0.0253968         0.968388          nan            0.867359       
0           30          0.353108    0.884375       0.247619          0.976874          nan            0.904819       
0           40          0.313793    0.91875        nan               0.969862          nan            0.944946       
0           50          0.285427    0.94375        nan               0.97709           nan            0.964531       
0           60          0.260501    0.928125       nan               0.973534          nan            0.951378       
1           70          0.273547    0.915625       nan               0.96385           nan            0.948145       


In [ ]:
# save
chainer.serializers.save_npz('./intern2019_resnet50_1.model', model)